In [1]:
try:
    import os
    import time
    import getpass
    import requests
    from arcgis.gis import GIS
    from arcgis.features import FeatureLayerCollection
    from arcgis.mapping import WebMap
    
    print('Modules loaded successfully')
    
except ImportError as e:
    print(f"Error importing module: {e}")

Modules Loaded


In [2]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    try:
        # prompt the user to enter their AGOL username
        username = input("Enter your ArcGIS Online username:")

        # prompt the user to enter their AGOL password (without echoing)
        password = getpass.getpass("Enter your ArcGIS Online password:")

        # create a GIS object and login to AGOL
        gis = GIS("https://www.arcgis.com", username, password)

        # confirm the login was successful
        if gis.users.me is not None:
            print ("Login successful. Logged in as:", gis.users.me.username)
        else:
            print ("Log in failed. Please check credientials.")
            return None
    
        # return the object
        return gis
    
    except Exception as e:
        print ("An error occurred during login:", e)
        return None
    
# run the login function

gis = login_AGOL()

Enter your ArcGIS Online username:Stacey_Vernon_GES23
Enter your ArcGIS Online password:········
Login successful. Logged in as: Stacey_Vernon_GES23


In [11]:
# Function to publish GeoJSON data to AGOL
def publish_geojson_to_agol(gis):
    try:
        BikeStationsZone_data = None

        # Search for existing feature layers with the title "BikeStations_Zones"
        zoneSearch = gis.content.search("title:BikeStations_Zones", item_type="Feature Layer")

        # If no existing feature layer is found, create a new one
        if not zoneSearch:
            # Set up the feature service properties in AGOL
            item_properties = {
                "title": "BikeStations_Zones",
                "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes",
                "tags": "dublinBikes",
                "type": "GeoJson"  # Note: Ensure this is the correct type, usually 'Feature Layer' is used for publishing
            }

            # URL link to the GeoJSON file
            geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
            geojson_item = gis.content.add(item_properties, geojson_url)

            # Publish the GeoJSON as a feature layer
            BikeStationsZone_data = geojson_item.publish()
            print("Data published. New feature layer created.")
            
        else:
            # Handle the case where the feature layer already exists
            BikeStationsZone_data = zoneSearch[0]
            print("Bike Station and Zone data already exists in AGOL.")
        
        return BikeStationsZone_data

    except Exception as e:
        print("An error occurred during publishing:", e)
        return None

# publish the geojso
BikeStationsZone_data = publish_geojson_to_agol(gis)

Bike Station and Zone data already exists in AGOL.


In [13]:
# URL to API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/bikes/bleeper_bikes/current/bikes.geojson"

def fetch_bike_data(url, max_retries=5, retry_delay=60):
    """
    Fetches bike data from the API endpoint with retries on failure.

    Parameters:
        url (str): The URL of the API endpoint.
        max_retries (int): Maximum number of retries before giving up.
        retry_delay (int): Delay between retries in seconds.

    Returns:
        dict or None: The bike data if successful, None otherwise.
    """
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url)
            if response.status_code in [200, 201]:
                # Parse the JSON response
                bike_data = response.json()
                return bike_data  # Data retrieval successful
            else:
                print(f'Failed to retrieve data. Status code: {response.status_code}. Retrying in {retry_delay} seconds...')
        except requests.RequestException as e:
            print(f'An error occurred: {e}. Retrying in {retry_delay} seconds...')
        
        # Increment attempt counter and wait before retrying
        attempt += 1
        time.sleep(retry_delay)

    return None  # Data retrieval failed after retries

# Fetch the bike data
bike_data = fetch_bike_data(bike_url)

# Output message based on data retrieval success or failure
if bike_data:
    print('Data retrieved successfully.')
else:
    print('No data retrieved, cannot proceed with further actions.')


Data retrieved successfully.


In [15]:
# create a FLC from the GeoJSON

# set up the feature service properties in AGOL for the stations and zone dataset
item_properties = {
    "title": "Bike_Locations_v2", #provide a title for your item
    "description": "Dublin Bike current locations", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }
    
# search AGOL contents to check if the layer already exists
fl_search = gis.content.search("title:Bike_Locations",item_type="Feature Layer")

if not fl_search:
    # if search list is empty (meaning a feature layer with that title does not exist), upload the GeoJSON 
    # & publish as a new feature service
    geojson_item = gis.content.add(item_properties, bike_url)
    live_bike_fl = geojson_item.publish()
    print("Successfully created new feature service and published live bike location")
    
else: 
    # if search list is NOT empty (meaning feature layer with that title already exists), overwrite the data within the FL
    live_data_fl = fl_search[0]
    content = FeatureLayerCollection.fromitem(live_data_fl)
    
    try:
        # overwriting the FLC data with new data
        overwrite_result = content.manager.overwrite(bike_url)

        # overwrite_result will have {"success":True} or {"success":False} - check the result
        if not overwrite_result['success']:
            print("Failed to overwrite Feature Layer Collection with new GeoJSON Data")
        else:
            print("Successfully updated Feature Layer Collection with new GeoJSON Data")
        
    except Exception as e:
        print("An error occurred during the process: {e}")

Successfully created new feature service and published live bike location


In [22]:
#  Search AGOL content to see if a map has already been created

# the title we are searching for
map_title = "DublinBike_LocationMap"

# searching in AGOL for these maps
existing_maps = gis.content.search(query=f"title:'{map_title}'", item_type = "Web Map")

if existing_maps:
    # retrieve the existing web map item
    web_map_item = existing_maps[0]
    web_map = WebMap(web_map_item)
    print ("Map already exists within AGOL Content")

else:
    print("No existing map found")

Map already exists within AGOL Content


In [28]:
if existing_maps:
    # Ask user if they want to update the map
    user_input = input("Do you wish to update the map? (Y/N): ").strip().upper()
    
    if user_input == 'Y':
        # update the map with layers defined but first check if layers are already in the map before adding
        layers_to_add = [live_bike_fl, BikeStationsZone_data]
        for layer in layers_to_add:
            if not any(existing_layer.id == layer.id for existing_layer in web_map.layers):
                web_map.add_layer(layer)
                
        web_map.update()
        web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
        print(f"Webmap updated successfully. Access the map here: {web_map_url}")
        display(web_map)
        
    else:
        # user has no updates so construct a URL to access the map
        web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
        print(f"No updates required. Access the existing map here: {web_map_url}")
        display(web_map)

else: 
    # if map doesn't exist, create one
    web_map = WebMap()
    web_map.add_layer(layer=live_bike_fl)
    web_map.add_layer(BikeStationsZone_data)
    web_map_properties = {
        "title": map_title,
        "description": "A web map of live Dublin Bike locations and their zones/stations obtained from Smart Dublin API",
        "tags": "Dublin Bike, locations, live webmap",
        "snippet": "A webmap of Dublin Bike locations"
    }
    
    web_map_item = web_map.save(item_properties=web_map_properties)
    web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
    print(f"New web map created and saved. Access it here: {web_map_url}")
    
    try:
        display(web_map)

    except ImportError:
        print("Display functionality not available in this environment.")

Do you wish to update the map? (Y/N): n
No updates required. Access the existing map here: https://www.arcgis.com/home/webmap/viewer.html?webmap=5f4db8353a1f4b10bf6a8578b9b61372


MapView(hide_mode_switch=True, jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=T…

In [25]:
# need to do:
# improve commenting 
# improve webmap - symbology?
# product heatmap of current locations - deduce busy areas??